In [1]:
import argparse
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve
)

In [3]:
!pip install ucimlrepo


In [4]:
from ucimlrepo import fetch_ucirepo

# Load Dataset from UCI

In [5]:
bank_marketing = fetch_ucirepo(id=222)

X = bank_marketing.data.features
y = bank_marketing.data.targets['y']
y = y.map({'yes': 1, 'no': 0}).astype(int)

In [18]:
# Target column is 'y'

# Identify Columns

In [6]:
categorical_cols = X.select_dtypes(include=["object"]).columns
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns

# Preprocessing

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_cols),
        ("num", StandardScaler(), numeric_cols)
    ]
)


# Model

In [8]:
model = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

# Train-Test Split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

# Train Model

In [10]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')),
                                                 ('num', StandardScaler(),
                                                  Index(['age', 'balance', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous'],
      dtype='object'))])),
                ('classifier', LogisticRegression(max_iter=1000))])

# Predict Probabilities

In [11]:
y_prob = model.predict_proba(X_test)[:, 1]

# Evaluation Function

In [12]:
def evaluate(threshold):
    y_pred = (y_prob >= threshold).astype(int)
    cm = confusion_matrix(y_test, y_pred)

    tn, fp, fn, tp = cm.ravel()

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    sensitivity = recall
    specificity = tn / (tn + fp)

    return cm, precision, recall, f1, sensitivity, specificity

# Evaluation @ Threshold 0.5

In [13]:
cm_05, p_05, r_05, f1_05, sens_05, spec_05 = evaluate(0.5)

print("=== Threshold = 0.5 ===")
print("Confusion Matrix:\n", cm_05)
print(f"Precision: {p_05:.4f}")
print(f"Recall (Sensitivity): {r_05:.4f}")
print(f"F1-score: {f1_05:.4f}")
print(f"Specificity: {spec_05:.4f}")

=== Threshold = 0.5 ===
Confusion Matrix:
 [[9726  255]
 [ 860  462]]
Precision: 0.6444
Recall (Sensitivity): 0.3495
F1-score: 0.4532
Specificity: 0.9745


# ROC-AUC

In [14]:
roc_auc = roc_auc_score(y_test, y_prob)
print(f"\nROC-AUC Score: {roc_auc:.4f}")


ROC-AUC Score: 0.9060


# Optimized Threshold (Max F1)

In [15]:
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

best_threshold = 0
best_f1 = 0

for t in thresholds:
    y_temp = (y_prob >= t).astype(int)
    f1_temp = f1_score(y_test, y_temp)
    if f1_temp > best_f1:
        best_f1 = f1_temp
        best_threshold = t

cm_opt, p_opt, r_opt, f1_opt, sens_opt, spec_opt = evaluate(best_threshold)

print("\n=== Optimized Threshold ===")
print(f"Best Threshold: {best_threshold:.4f}")
print("Confusion Matrix:\n", cm_opt)
print(f"Precision: {p_opt:.4f}")
print(f"Recall (Sensitivity): {r_opt:.4f}")
print(f"F1-score: {f1_opt:.4f}")
print(f"Specificity: {spec_opt:.4f}")


=== Optimized Threshold ===
Best Threshold: 0.2216
Confusion Matrix:
 [[9228  753]
 [ 463  859]]
Precision: 0.5329
Recall (Sensitivity): 0.6498
F1-score: 0.5855
Specificity: 0.9246


# Save probabilities.csv

In [16]:
output_df = pd.DataFrame({
    "RecordId": X_test.index,
    "Probability(yes)": y_prob,
    "PredictedLabel": (y_prob >= best_threshold).astype(int)
})

output_df.to_csv("probabilities.csv", index=False)

print("\nprobabilities.csv saved successfully")


probabilities.csv saved successfully


Why ROC Curve is Useful

ROC curve shows performance across all classification thresholds

Plots True Positive Rate vs False Positive Rate

Works well even when classes are imbalanced

ROC-AUC summarizes model’s discrimination ability

 Higher ROC-AUC = better separation between “yes” and “no”

What Changes When Threshold Changes (Precision–Recall Trade-off)
Threshold	Precision	Recall
Lower	↓	↑
Higher	↑	↓

Lower threshold → more positives predicted → high recall

Higher threshold → stricter prediction → high precision

Business decision decides optimal threshold

 That’s why accuracy alone is not sufficient